In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.18.0


In [6]:
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_CLASSES = 38 
DATA_DIR = '../input/plantvillage'

print(f"Image size set to: {IMAGE_SIZE}")
print(f"Data directory is: {DATA_DIR}")

Image size set to: (256, 256)
Data directory is: ../input/plantvillage


In [8]:
train_dir = '../input/plantvillage/PlantVillage/train' #training data path
val_dir = '../input/plantvillage/PlantVillage/val' #validation data path

#training dataset 
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, 
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

#validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,  
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Loaded {len(class_names)} classes.")
print(f"First 5 classes: {class_names[:5]}...")

Found 43444 files belonging to 38 classes.
Found 10861 files belonging to 38 classes.
Loaded 38 classes.
First 5 classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy']...


In [3]:
!find ../input/plantvillage/PlantVillage/train -type f | wc -l # good news it is already segmented to check ki kitni files hai
!find ../input/plantvillage/PlantVillage/val -type f | wc -l 

43444
10861


In [9]:
data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomShear(x_factor=(0.0, 0.2))
  ],
  name="data_augmentation",
)

In [10]:
#Load the VGG16 base model (pre-trained on ImageNet)
base_model = tf.keras.applications.VGG16(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False, 
    weights='imagenet'
)

# Freeze the base model
base_model.trainable = False

#Create your new model on top
model_vgg16 = Sequential([
    # Input layer
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    
    # Rescale pixel values from [0, 255] to [0, 1] as mentioned in the paper
    layers.Rescaling(1./255),
    
    # Apply the data augmentation
    data_augmentation,
    
    # The VGG16 base
    base_model,
    
    # Flatten the output to feed into our classifier
    layers.GlobalAveragePooling2D(),
    
    # Our custom classifier head
    # The paper used a Dense layer and softmax activation
    layers.Dense(NUM_CLASSES, activation='softmax') 
], name="VGG16_Transfer_Learning")

# Compile the model with the Adam optimizer and categorical cross-entropy loss
model_vgg16.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_vgg16.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "VGG16_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 8, 8, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        19,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,734,182 (56.21 MB)

 Trainable params: 19,494 (76.15 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
tf.keras.backend.clear_session()

In [13]:
# Train the model
epochs_to_run = 20

history = model_vgg16.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_to_run
)

Epoch 1/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 303s 223ms/step - accuracy: 0.3671 - loss: 2.5318 - val_accuracy: 0.6696 - val_loss: 1.3980
Epoch 2/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 297s 219ms/step - accuracy: 0.6924 - loss: 1.3099 - val_accuracy: 0.7576 - val_loss: 0.9930
Epoch 3/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 296s 218ms/step - accuracy: 0.7620 - loss: 0.9791 - val_accuracy: 0.7917 - val_loss: 0.8169
Epoch 4/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 296s 218ms/step - accuracy: 0.7968 - loss: 0.8198 - val_accuracy: 0.8088 - val_loss: 0.7161
Epoch 5/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 296s 218ms/step - accuracy: 0.8161 - loss: 0.7199 - val_accuracy: 0.8263 - val_loss: 0.6449
Epoch 6/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 296s 218ms/step - accuracy: 0.8307 - loss: 0.6494 - val_accuracy: 0.8345 - val_loss: 0.5999
Epoch 7/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 296s 218ms/step - accuracy: 0.8403 - loss: 0.6064 - val_accuracy: 0.8459 - val_loss: 0.5574
Epoch 8/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 297s 218ms/step - ac

In [11]:
model_vgg16.save("/kaggle/working/plant_model_vgg16.keras")
print("Model saved successfully!")

Model saved successfully!


In [6]:
# Custom wrapper to handle the old format
class FixedRandomShear(keras.layers.RandomShear):
    @classmethod
    def from_config(cls, config):
        # Fix negative factors
        if 'x_factor' in config:
            x_factor = config['x_factor']
            if isinstance(x_factor, list) and len(x_factor) == 2:
                config['x_factor'] = [abs(x_factor[0]), abs(x_factor[1])]
        return super().from_config(config)

# Load your existing model
model = keras.models.load_model(
    '/kaggle/input/ai-vgg-16/keras/default/1/plant_model_vgg16.h5', 
    custom_objects={'RandomShear': FixedRandomShear}
)


In [7]:
model.summary()

Model: "VGG16_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 8, 8, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        19,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,734,184 (56.21 MB)

 Trainable params: 19,494 (76.15 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)

In [10]:
loss, accuracy = model.evaluate(val_ds, steps=100) 
print(f"Model loss: {loss}, Accuracy: {accuracy}")

100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 151ms/step - accuracy: 0.8704 - loss: 0.4154
Model loss: 0.4230698347091675, Accuracy: 0.8671875


In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.0001), # Use the same low learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model re-compiled.")

epochs_to_run = 5 
history_continued = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_to_run
)

Model re-compiled.
Epoch 1/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 292s 212ms/step - accuracy: 0.8739 - loss: 0.4524 - val_accuracy: 0.8694 - val_loss: 0.4351
Epoch 2/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 283s 209ms/step - accuracy: 0.8743 - loss: 0.4479 - val_accuracy: 0.8703 - val_loss: 0.4352
Epoch 3/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 283s 208ms/step - accuracy: 0.8726 - loss: 0.4486 - val_accuracy: 0.8697 - val_loss: 0.4338
Epoch 4/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 283s 208ms/step - accuracy: 0.8725 - loss: 0.4463 - val_accuracy: 0.8701 - val_loss: 0.4337
Epoch 5/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 283s 209ms/step - accuracy: 0.8770 - loss: 0.4407 - val_accuracy: 0.8702 - val_loss: 0.4328


# Fine tunning..

In [14]:
vgg_base_layer = model.get_layer('vgg16') 

# Unfreeze it
vgg_base_layer.trainable = True

print("VGG16 layer within the loaded model unfrozen.")

model.summary()

VGG16 layer within the loaded model unfrozen.


Model: "VGG16_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 8, 8, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        19,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,773,172 (56.36 MB)

 Trainable params: 14,734,182 (56.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 38,990 (152.31 KB)

In [15]:
# Re-compiling the model with a very low learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5), # Using 0.00001
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model re-compiled for fine-tuning.")

Model re-compiled for fine-tuning.


In [16]:
fine_tune_epochs = 5
previous_total_epochs = 20 
total_epochs = previous_total_epochs + fine_tune_epochs 

history_fine_tune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs, 
    initial_epoch=previous_total_epochs 
                                        
)

Epoch 21/25
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 870s 630ms/step - accuracy: 0.9008 - loss: 0.3091 - val_accuracy: 0.9291 - val_loss: 0.2238
Epoch 22/25
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 852s 628ms/step - accuracy: 0.9583 - loss: 0.1296 - val_accuracy: 0.9592 - val_loss: 0.1241
Epoch 23/25
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 853s 628ms/step - accuracy: 0.9731 - loss: 0.0826 - val_accuracy: 0.9477 - val_loss: 0.1570
Epoch 24/25
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 853s 628ms/step - accuracy: 0.9797 - loss: 0.0655 - val_accuracy: 0.9382 - val_loss: 0.2076
Epoch 25/25
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 852s 628ms/step - accuracy: 0.9824 - loss: 0.0531 - val_accuracy: 0.9750 - val_loss: 0.0753


In [17]:
model.save("/kaggle/working/plant_model_vgg16.h5") 
print("Model saved successfully!")

Model saved successfully!
